In [ ]:
import lightgbm as lgb
from art.estimators.classification import LightGBMClassifier
from art.utils import load_mnist
import numpy as np
from art.attacks.evasion import ZooAttack
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

In [ ]:
num_test_data = 10
X_test = X_test[:num_test_data]
y_test = y_test[:num_test_data]

In [ ]:
nb_sample_train = X_train.shape[0]
nb_sample_test = X_test.shape[0]
X_train = X_train.reshape((nb_sample_train, 28 * 28))
X_test = X_test.reshape((nb_sample_test, 28 * 28))

In [ ]:
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': 10,
}
lgb_train = lgb.Dataset(X_train, label=np.argmax(y_train, axis=1))
lgb_test = lgb.Dataset(X_test, label=np.argmax(y_test, axis=1))
model = lgb.train(params=params, train_set=lgb_train, num_boost_round=100, valid_sets=[lgb_test])

In [ ]:
victim_classifier = LightGBMClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value))

In [ ]:
attack = ZooAttack(
    classifier=victim_classifier,
    confidence=0.5,
    learning_rate=1e-1,
    max_iter=200,
    binary_search_steps=100,
    initial_const=1e-1,
    nb_parallel=250,
    batch_size=1,
    variable_h=0.01
)

In [ ]:
X_test_adv = attack.generate(x=X_test)

In [ ]:
preds = victim_classifier.predict(X_test)
preds_adv = victim_classifier.predict(X_test_adv)

print('test data before attack:', np.argmax(preds, axis=1))
print('test data after attack:', np.argmax(preds_adv, axis=1))

In [ ]:
target_index = 4
print((f'before: {np.argmax(preds[target_index])}, after: {np.argmax(preds_adv[target_index])}'))
print('l2', np.linalg.norm((X_test[target_index] - X_test_adv[target_index])).mean())


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(X_test[target_index].reshape((28, 28)), cmap='gray')
ax[1].imshow(X_test_adv[target_index].reshape((28, 28)), cmap='gray')
plt.show()